In [4]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

In [1]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

In [3]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [5]:
from dotenv import load_dotenv
import os

# loads variables from .env
load_dotenv()

True

In [7]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY is None:
    raise ValueError("Environment variable GOOGLE_API_KEY not found!")

print("Secret loaded successfully!")

Secret loaded successfully!


In [9]:
client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Explain AI to me like I'm a kid.")

print(response.text)

Okay, imagine you have a really, really smart puppy. This puppy can learn things, but you have to teach it first.

That puppy is like AI, or Artificial Intelligence. "Artificial" means not real, like a fake tree. "Intelligence" means being able to learn and understand things, like you learning how to ride your bike!

So, AI is like a brain that's not inside a real person or animal. It's made with computers!

Here's how it works:

*   **You show it lots of examples:** Imagine you want to teach the puppy to recognize a cat. You show it hundreds of pictures of cats.
*   **It learns the patterns:** The puppy looks at all those pictures and figures out what makes a cat a cat: pointy ears, whiskers, a tail, etc. The computer does this too!
*   **It can do new things:** Now, when you show the puppy a new picture, it can say, "That's a cat!" even if it's never seen that specific cat before. The computer can do the same with new information it's given.

AI can do lots of cool things:

*   **Hel

In [10]:
Markdown(response.text)

Okay, imagine you have a really, really smart puppy. This puppy can learn things, but you have to teach it first.

That puppy is like AI, or Artificial Intelligence. "Artificial" means not real, like a fake tree. "Intelligence" means being able to learn and understand things, like you learning how to ride your bike!

So, AI is like a brain that's not inside a real person or animal. It's made with computers!

Here's how it works:

*   **You show it lots of examples:** Imagine you want to teach the puppy to recognize a cat. You show it hundreds of pictures of cats.
*   **It learns the patterns:** The puppy looks at all those pictures and figures out what makes a cat a cat: pointy ears, whiskers, a tail, etc. The computer does this too!
*   **It can do new things:** Now, when you show the puppy a new picture, it can say, "That's a cat!" even if it's never seen that specific cat before. The computer can do the same with new information it's given.

AI can do lots of cool things:

*   **Help you find videos:** Like when you ask your tablet to play your favorite cartoon.
*   **Play games:** Some video game characters are controlled by AI.
*   **Help doctors find diseases:** AI can look at lots of medical information to help doctors figure out what's wrong with people.
*   **Drive cars (someday!):** Some cars are learning to drive themselves using AI.

So, AI is like a super-smart computer that can learn and do cool things, like recognize cats, play games, and maybe even drive your car one day!


## Multi-turn conversations

In [11]:
chat = client.chats.create(model='gemini-2.0-flash', history=[])
response = chat.send_message('Hello! My name is Zlork.')
print(response.text)

Greetings, Zlork! It's nice to meet you. Is there anything I can help you with today?



In [12]:
response = chat.send_message('Can you tell me something interesting about dinosaurs?')
print(response.text)

Okay, here's a fascinating fact about dinosaurs:

**Some dinosaurs, like the Microraptor, may have had four wings!**

Microraptor was a small, feathered dinosaur that lived during the Early Cretaceous period. Fossils show that it had feathers on both its forelimbs and hindlimbs, forming what are essentially two pairs of wings. While it probably couldn't fly as efficiently as modern birds, it likely used these four wings for gliding between trees and maneuvering in the air.

Isn't that wild? A four-winged dinosaur! It really challenges our traditional image of what dinosaurs were like.



In [13]:
response = chat.send_message('Do you remember what my name is?')
print(response.text)

Yes, Zlork. I remember your name is Zlork.



## Checking available models

In [14]:
for model in client.models.list():
  print(model.name)

models/embedding-gecko-001
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-

In [15]:
from pprint import pprint

for model in client.models.list():
  if model.name == 'models/gemini-2.0-flash':
    pprint(model.to_json_dict())
    break

{'description': 'Gemini 2.0 Flash',
 'display_name': 'Gemini 2.0 Flash',
 'input_token_limit': 1048576,
 'name': 'models/gemini-2.0-flash',
 'output_token_limit': 8192,
 'supported_actions': ['generateContent',
                       'countTokens',
                       'createCachedContent',
                       'batchGenerateContent'],
 'tuned_model_info': {},
 'version': '2.0'}


## Explore generation parameters 

In [43]:
from google.genai import types

# Here we limit the amount of output tokens
short_config = types.GenerateContentConfig(max_output_tokens=200)

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents='Write a 200 word essay on the importance of olives in modern society.')

print(response.text)

The humble olive, a fruit cultivated for millennia, continues to hold a significant place in modern society. Beyond its Mediterranean origins, the olive and its derivatives, particularly olive oil, are now staples in global cuisine and contribute significantly to diverse sectors.

Olive oil is celebrated for its health benefits, lauded for its monounsaturated fats and antioxidants. Its role in the Mediterranean diet, associated with longevity and reduced risk of chronic diseases, has solidified its position as a healthy cooking oil. This has driven increased demand, fueling a thriving industry that spans from agriculture and processing to export and distribution.

Furthermore, olives themselves are a versatile food. Eaten whole, stuffed, or incorporated into dishes, they add a unique briny flavor appreciated worldwide. They feature prominently in tapas, salads, and numerous Mediterranean and Middle Eastern recipes, highlighting their adaptability across diverse culinary traditions.

Be

In [45]:
response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents='Write a short poem on the importance of olives in modern society.')

print(response.text)

From groves of sun, a gift descends,
The humble olive, modern trends.
In oil that gleams, a healthy pour,
On salads bright, and sauces' core.

Tapenade's zest, a pizza's art,
A global flavor, set apart.
No longer just a Grecian dream,
But woven in our culinary scheme.

So raise a glass, to salty bite,
The olive's reign, both dark and bright.
A symbol strong, of peace and taste,
A modern marvel, never waste.



### Temperature

Temperature controls the degree of randomness in token selection. Higher temperatures result in a higher number of candidate tokens from which the next output token is selected, and can produce more diverse results, while lower temperatures have the opposite effect, such that a temperature of 0 results in greedy decoding, selecting the most probable token at each step.

Temperature doesn't provide any guarantees of randomness, but it can be used to "nudge" the output somewhat.

In [85]:
high_temp_config = types.GenerateContentConfig(temperature=2.0)
# Temp 0.0 will always give Azure
# Temp 0.5 gives mostly Azure but sometimes other colors.
# Temp 2.0 gives almost always something different.

for _ in range(5):
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=high_temp_config,
      contents='Pick a random colour... (respond in a single word)')

  if response.text:
    print(response.text, '-' * 25)

Purple
 -------------------------
Turquoise
 -------------------------
Cerulean
 -------------------------
Turquoise
 -------------------------
Azure
 -------------------------


### Top-P

Like temperature, the top-P parameter is also used to control the diversity of the model's output.

Top-P defines the probability threshold that, once cumulatively exceeded, tokens stop being selected as candidates. A top-P of 0 is typically equivalent to greedy decoding, and a top-P of 1 typically selects every token in the model's vocabulary.

You may also see top-K referenced in LLM literature. Top-K is not configurable in the Gemini 2.0 series of models, but can be changed in older models. Top-K is a positive integer that defines the number of most probable tokens from which to select the output token. A top-K of 1 selects a single token, performing greedy decoding.

Run this example a number of times, change the settings and observe the change in output.

In [108]:
model_config = types.GenerateContentConfig(
    # These are the default values for gemini-2.0-flash.
    temperature=1.0,
    top_p=0.95,
)

story_prompt = "You are a creative writer. Write a short story about a cat who goes on an adventure."
response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=model_config,
    contents=story_prompt,
)
print(response.text)

Clementine was, by all accounts, a pampered Persian. She lived in a sun-drenched apartment filled with plush velvet pillows, a scratching post shaped like a miniature Eiffel Tower, and a human named Eleanor who catered to her every whim. But Clementine yearned for more than tuna pate and feather wands. She yearned for adventure.

One particularly dull afternoon, while Eleanor was engrossed in a book about Victorian embroidery, Clementine spotted it: a crack in the living room window, a sliver of the outside world beckoning. A world of rustling leaves, chirping birds, and… possibilities.

With a burst of unexpected agility, Clementine squeezed through the opening. The cool air nipped at her fur, carrying the scent of damp earth and something tantalizingly unknown. This was it. Her adventure.

Her first challenge was the fire escape. The metal was cold and unforgiving, but Clementine, fueled by her newfound freedom, navigated it with surprising grace. She descended, her fluffy tail actin

_Temperature scales the probabilities of all potential next words, making extreme choices more or less likely, controlling overall creativity. In contrast, Top-P dynamically selects a subset of tokens (the "nucleus") whose cumulative probabilities add up to 'P', focusing generation on the most confident, relevant words while cutting off less probable, often irrelevant, tails. Use Temperature for broad creativity/conservatism, and Top-P for dynamically adjusting the pool of acceptable words for coherent diversity._

## Prompting

### Zero-shot

Zero-shot prompts are prompts that describe the request for the model directly.


In [175]:
model_config = types.GenerateContentConfig(
    temperature=0.1,
    top_p=1,
    max_output_tokens=5,
)

zero_shot_prompt = """Classify movie reviews as POSITIVE, NEUTRAL or NEGATIVE.
Review: "Her" is a disturbing study revealing the direction
humanity is headed if AI is allowed to keep evolving,
unchecked. We should be cautiously optimistic.
Sentiment: """

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=model_config,
    contents=zero_shot_prompt)

print(response.text)

NEGATIVE



#### Enum mode

The models are trained to generate text, and while the Gemini 2.0 models are great at following instructions, other models can sometimes produce more text than you may wish for. In the preceding example, the model will output the label, but sometimes it can include a preceding "Sentiment" label, and without an output token limit, it may also add explanatory text afterwards. See [this prompt in AI Studio](https://aistudio.google.com/prompts/1gzKKgDHwkAvexG5Up0LMtl1-6jKMKe4g) for an example.

The Gemini API has an [Enum mode](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Enum.ipynb) feature that allows you to constrain the output to a fixed set of values.

In [177]:
import enum

class Sentiment(enum.Enum):
    POSITIVE = "positive"
    NEUTRAL = "neutral"
    NEGATIVE = "negative"


response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        response_mime_type="text/x.enum",
        response_schema=Sentiment
    ),
    contents=zero_shot_prompt)

print(response.text)

neutral


In [179]:
enum_response = response.parsed
print(enum_response)
print(type(enum_response))

Sentiment.NEUTRAL
<enum 'Sentiment'>


## One-Shot and Few-Shot
Providing an example of the expected response is known as a "one-shot" prompt. When you provide multiple examples, it is a "few-shot" prompt.

```
Classify the sentiment of the following text as positive, negative, or neutral.
Text: The product is terrible.
Sentiment: Negative

Text: I think the vacation was okay. Sentiment:
```
This is one shot as one example was provided.

few_shot_prompt = """Parse a customer's pizza order into valid JSON:

EXAMPLE:
I want a small pizza with cheese, tomato sauce, and pepperoni.
JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "pepperoni"]
}
```

EXAMPLE:
Can I get a large pizza with tomato sauce, basil and mozzarella
JSON Response:
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}
```

ORDER:
"""

customer_order = "Give me a large with cheese & pineapple"

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=250,
    ),
    contents=[few_shot_prompt, customer_order])

print(response.text)

#### JSON mode

To provide control over the schema, and to ensure that you only receive JSON (with no other text or markdown), you can use the Gemini API's [JSON mode](https://github.com/google-gemini/cookbook/blob/main/quickstarts/JSON_mode.ipynb). This forces the model to constrain decoding, such that token selection is guided by the supplied schema.

In [190]:
import typing_extensions as typing

class PizzaOrder(typing.TypedDict):
    size: str
    ingredients: list[str]
    type: str


response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        response_mime_type="application/json",
        response_schema=PizzaOrder,
    ),
    contents="Can I have a large dessert pizza with apple and chocolate")

print(response.text)

{
  "size": "large",
  "ingredients": ["apple", "chocolate"],
  "type": "dessert"
}


### Chain of Thought (CoT)

Direct prompting on LLMs can return answers quickly and (in terms of output token usage) efficiently, but they can be prone to hallucination. The answer may "look" correct (in terms of language and syntax) but is incorrect in terms of factuality and reasoning.

Chain-of-Thought prompting is a technique where you instruct the model to output intermediate reasoning steps, and it typically gets better results, especially when combined with few-shot examples. It is worth noting that this technique doesn't completely eliminate hallucinations, and that it tends to cost more to run, due to the increased token count.

Models like the Gemini family are trained to be "chatty" or "thoughtful" and will provide reasoning steps without prompting, so for this simple example you can ask the model to be more direct in the prompt to force a non-reasoning response. Try re-running this step if the model gets lucky and gets the answer correct on the first try.

In [195]:
prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
am 20 years old. How old is my partner? Return the answer directly."""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt)

print(response.text)

52



The expected answer above is 28. When they were 4 years old, the partner was 3*4 = 12, an 8 year difference.
Now the person is 20 years old, so the partner must be 28 years old.

In [198]:
prompt = """When I was 4 years old, my partner was 3 times my age. Now,
I am 20 years old. How old is my partner? Let's think step by step."""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt)

Markdown(response.text)

Here's how to solve the problem step-by-step:

1.  **Find the age difference:** When you were 4, your partner was 3 times your age, so they were 4 * 3 = 12 years old.

2.  **Calculate the age difference:** The age difference between you and your partner is 12 - 4 = 8 years.

3.  **Determine your partner's current age:** Since you are now 20 and your partner is 8 years older, they are currently 20 + 8 = 28 years old.

**Answer:** Your partner is 28 years old.

### ReAct: Reason and act

In this example you will run a ReAct prompt directly in the Gemini API and perform the searching steps yourself. As this prompt follows a well-defined structure, there are frameworks available that wrap the prompt into easier-to-use APIs that make tool calls automatically, such as the LangChain example from the "Prompting" whitepaper.

To try this out with the Wikipedia search engine, check out the [Searching Wikipedia with ReAct](https://github.com/google-gemini/cookbook/blob/main/examples/Search_Wikipedia_using_ReAct.ipynb) cookbook example.


> Note: The prompt and in-context examples used here are from [https://github.com/ysymyth/ReAct](https://github.com/ysymyth/ReAct) which is published under an [MIT license](https://opensource.org/licenses/MIT), Copyright (c) 2023 Shunyu Yao.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/18oo63Lwosd-bQ6Ay51uGogB3Wk3H8XMO">Open in AI Studio</a>
  </td>
</table>


In [204]:
model_instructions = """
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation,
Observation is understanding relevant information from an Action's output and Action can be one of three types:
 (1) <search>entity</search>, which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it
     will return some similar entities to search and you can try to search the information from those topics.
 (2) <lookup>keyword</lookup>, which returns the next sentence containing keyword in the current context. This only does exact matches,
     so keep your searches short.
 (3) <finish>answer</finish>, which returns the answer and finishes the task.
"""

example1 = """Question
Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?

Thought 1
The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.

Action 1
<search>Milhouse</search>

Observation 1
Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.

Thought 2
The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".

Action 2
<lookup>named after</lookup>

Observation 2
Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.

Thought 3
Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.

Action 3
<finish>Richard Nixon</finish>
"""

example2 = """Question
What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?

Thought 1
I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.

Action 1
<search>Colorado orogeny</search>

Observation 1
The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought 2
It does not mention the eastern sector. So I need to look up eastern sector.

Action 2
<lookup>eastern sector</lookup>

Observation 2
The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought 3
The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.

Action 3
<search>High Plains</search>

Observation 3
High Plains refers to one of two distinct land regions

Thought 4
I need to instead search High Plains (United States).

Action 4
<search>High Plains (United States)</search>

Observation 4
The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130m).

Thought 5
High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.

Action 5
<finish>1,800 to 7,000 ft</finish>
"""

# Come up with more examples yourself, or take a look through https://github.com/ysymyth/ReAct/

In [206]:
question = """Question
Who was the youngest author listed on the transformers NLP paper?
"""

# You will perform the Action; so generate up to, but not including, the Observation.
react_config = types.GenerateContentConfig(
    stop_sequences=["\nObservation"],
    system_instruction=model_instructions + example1 + example2,
)

# Create a chat that has the model instructions and examples pre-seeded.
react_chat = client.chats.create(
    model='gemini-2.0-flash',
    config=react_config,
)

resp = react_chat.send_message(question)
print(resp.text)

Thought 1
I need to find the transformers NLP paper and identify the youngest author listed on it.

Action 1
<search>transformers NLP paper</search>



In [208]:
observation = """Observation 1
[1706.03762] Attention Is All You Need
Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely.
"""
resp = react_chat.send_message(observation)
print(resp.text)

Thought 2
The paper lists the authors Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin. I need to find the youngest among them. This would likely involve finding their birthdates, which would require multiple searches. I will start with the first author, Ashish Vaswani.

Action 2
<search>Ashish Vaswani</search>



## Thinking mode

The experiemental Gemini Flash 2.0 "Thinking" model has been trained to generate the "thinking process" the model goes through as part of its response. As a result, the Flash Thinking model is capable of stronger reasoning capabilities in its responses.

Using a "thinking mode" model can provide you with high-quality responses without needing specialised prompting like the previous approaches. One reason this technique is effective is that you induce the model to generate relevant information ("brainstorming", or "thoughts") that is then used as part of the context in which the final response is generated.

Note that when you use the API, you get the final response from the model, but the thoughts are not captured. To see the intermediate thoughts, try out [the thinking mode model in AI Studio](https://aistudio.google.com/prompts/new_chat?model=gemini-2.0-flash-thinking-exp-01-21).

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1Z991SV7lZZZqioOiqIUPv9a9ix-ws4zk">Open in AI Studio</a>
  </td>
</table>

In [228]:
import io
from IPython.display import Markdown, clear_output


response = client.models.generate_content_stream(
    model='gemini-2.0-flash-thinking-exp',
    contents='Who was the youngest author listed on the transformers NLP paper?',
)

buf = io.StringIO()
for chunk in response:
    buf.write(chunk.text)
    # Display the response as it is streamed
    print(chunk.text, end='')

# And then render the finished response as formatted markdown.
clear_output()
Markdown(buf.getvalue())

The youngest author listed on the "Attention Is All You Need" paper (which introduced the Transformer model) was **Aidan N. Gomez**.

He was 20 years old when the paper was published in June 2017. At the time, he was a PhD student at the University of Toronto.

## Code Prompting

In [231]:
# The Gemini models love to talk, so it helps to specify they stick to the code if that
# is all that you want.
code_prompt = """
Write a Python function to calculate the factorial of a number. No explanation, provide only the code.
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=1,
        top_p=1,
        max_output_tokens=1024,
    ),
    contents=code_prompt)

Markdown(response.text)

```python
def factorial(n):
  """Calculate the factorial of a number."""
  if n == 0:
    return 1
  else:
    return n * factorial(n-1)
```

In [238]:
# See the markdown format
response.text

'```python\ndef factorial(n):\n  """Calculate the factorial of a number."""\n  if n == 0:\n    return 1\n  else:\n    return n * factorial(n-1)\n```'

## Code Execution

The Gemini API can automatically run generated code too, and will return the output.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/11veFr_VYEwBWcLkhNLr-maCG0G8sS_7Z">Open in AI Studio</a>
  </td>
</table>

In [245]:
from pprint import pprint

config = types.GenerateContentConfig(
    tools=[types.Tool(code_execution=types.ToolCodeExecution())],
)

code_exec_prompt = """
Generate the first 14 odd prime numbers, then calculate their sum.
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=config,
    contents=code_exec_prompt)

for part in response.candidates[0].content.parts:
  pprint(part.to_json_dict())
  print("-----")

{'text': 'Okay, I can do that. First, I need to generate the first 14 odd '
         'prime numbers. A prime number is a number greater than 1 that has '
         'only two factors: 1 and itself. Odd prime numbers are prime numbers '
         'that are not divisible by 2.\n'
         '\n'
         "Here's how I'll approach this: I'll start listing odd numbers and "
         'checking if they are prime.\n'
         '\n'
         '1.  3 is prime.\n'
         '2.  5 is prime.\n'
         '3.  7 is prime.\n'
         '4.  9 is not prime (3\\*3).\n'
         '5.  11 is prime.\n'
         '6.  13 is prime.\n'
         '7.  15 is not prime (3\\*5).\n'
         '8.  17 is prime.\n'
         '9.  19 is prime.\n'
         '10. 21 is not prime (3\\*7).\n'
         '11. 23 is prime.\n'
         '12. 25 is not prime (5\\*5).\n'
         '13. 27 is not prime (3\\*9).\n'
         '14. 29 is prime.\n'
         '15. 31 is prime.\n'
         '16. 33 is not prime (3\\*11).\n'
         '17. 35 is not prim

This response contains multiple parts, including an opening and closing text part that represent regular responses, an `executable_code` part that represents generated code and a `code_execution_result` part that represents the results from running the generated code.

You can explore them individually.

In [249]:
for part in response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))
    elif part.executable_code:
        display(Markdown(f'```python\n{part.executable_code.code}\n```'))
    elif part.code_execution_result:
        if part.code_execution_result.outcome != 'OUTCOME_OK':
            display(Markdown(f'## Status {part.code_execution_result.outcome}'))

        display(Markdown(f'```\n{part.code_execution_result.output}\n```'))

Okay, I can do that. First, I need to generate the first 14 odd prime numbers. A prime number is a number greater than 1 that has only two factors: 1 and itself. Odd prime numbers are prime numbers that are not divisible by 2.

Here's how I'll approach this: I'll start listing odd numbers and checking if they are prime.

1.  3 is prime.
2.  5 is prime.
3.  7 is prime.
4.  9 is not prime (3\*3).
5.  11 is prime.
6.  13 is prime.
7.  15 is not prime (3\*5).
8.  17 is prime.
9.  19 is prime.
10. 21 is not prime (3\*7).
11. 23 is prime.
12. 25 is not prime (5\*5).
13. 27 is not prime (3\*9).
14. 29 is prime.
15. 31 is prime.
16. 33 is not prime (3\*11).
17. 35 is not prime (5\*7).
18. 37 is prime.
19. 39 is not prime (3\*13).
20. 41 is prime.
21. 43 is prime.
22. 45 is not prime (5\*9).
23. 47 is prime.

So, the first 14 odd prime numbers are: 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47.

Now, let's calculate the sum of these numbers using python.



```python
import numpy as np
primes = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
sum_of_primes = np.sum(primes)
print(sum_of_primes)


```

```
326

```

Therefore, the sum of the first 14 odd prime numbers is 326.
